In [3]:
import pickle
import json
from shapely.geometry import shape

import networkx as nx
import osmnx as ox
import geopandas as gpd
import numpy as np

from my_paths import *

In [ ]:
PATH_STRADE_MOTORWAY = '..\\Data\\Raw\\Info_bici\\strade_motorway.geojson'
PATH_STRADE_TRUNK = '..\\Data\\Raw\\Info_bici\\strade_trunk.geojson'
PATH_STRADE_PRIMARY = '..\\Data\\Raw\\Info_bici\\strade_primary.geojson'
PATH_STRADE_PRIMARY_NULL = '..\\Data\\Raw\\Info_bici\\strade_primary_null.geojson'
PATH_STRADE_FOOTWAY = '..\\Data\\Raw\\Info_bici\\strade_footway.geojson'
PATH_STRADE_SECONDARY = "..\\Data\\Raw\\Info_bici\\strade_secondary.geojson"
PATH_STRADE_TERTIARY = "..\\Data\\Raw\\Info_bici\\strade_tertiary.geojson"
PATH_STRADE_RESIDENTIAL = "..\\Data\\Raw\\Info_bici\\strade_residential.geojson"
PATH_STRADE_SERVICE = "..\\Data\\Raw\\Info_bici\\strade_service.geojson"

PATH_highway_null_unclassified = "Data/Staging/Info_bici/highway_null_unclassified.geojson"

In [ ]:
with open(PATH_STRADE_CICLABILI_PICKLE_RAW, "rb") as f:
    G_strade = pickle.load(f)

In [ ]:
gdf = ox.graph_to_gdfs(G_strade, nodes=False)

In [ ]:
gdf.head
gdf["highway"].value_counts()

In [32]:
gdf_motorways = gdf[(gdf["highway"] == "motorway")]
len(gdf_motorways)

81

In [ ]:
gdf_motorways = gdf[(gdf["highway"] == "motorway") | (gdf["highway"] == "motorway_link")]

print(len(gdf_motorways))  # Mostra il numero di righe

# Salva in GeoJSON
gdf_motorways.to_file(PATH_STRADE_MOTORWAY, driver='GeoJSON')

311


In [ ]:
gdf_trunks = gdf[(gdf["highway"] == "trunk") | (gdf["highway"] == "trunk_link")]

print(len(gdf_trunks))  # Mostra il numero di righe

# Salva in GeoJSON
gdf_trunks.to_file(PATH_STRADE_TRUNK, driver='GeoJSON')

106


In [ ]:
gdf_primary = gdf[(gdf["highway"] == "primary") | (gdf["highway"] == "primary_link")]


print(len(gdf_primary))  # Mostra il numero di righe

# Salva in GeoJSON
gdf_primary.to_file(PATH_STRADE_PRIMARY, driver='GeoJSON')


In [ ]:
# Sostituisci i None con np.nan nel GeoDataFrame primario
gdf_primary.loc[:, "maxspeed"] = gdf_primary["maxspeed"].replace({None: np.nan})

# Filtra solo le righe con maxspeed mancante
gdf_null_maxspeed = gdf_primary[gdf_primary["maxspeed"].isna()]

# Salva su file GeoJSON
gdf_null_maxspeed.to_file(PATH_STRADE_PRIMARY_NULL, driver='GeoJSON')


In [ ]:
gdf_footway = gdf[(gdf["highway"] == "footway")]


print(len(gdf_footway))  # Mostra il numero di righe

# Salva in GeoJSON
gdf_primary.to_file(PATH_STRADE_FOOTWAY, driver='GeoJSON')


In [ ]:
# Strade Secondary

gdf_secondary = gdf[(gdf["highway"] == "secondary")]
len(gdf_secondary)
print(gdf[gdf["highway"] == "secondary"].isna().sum())

gdf_secondary = gdf[(gdf["highway"] == "secondary") | (gdf["highway"] == "secondary_link")]

print(len(gdf_secondary))  # Mostra il numero di righe

# Salva in GeoJSON
gdf_secondary.to_file(PATH_STRADE_SECONDARY, driver='GeoJSON')

In [ ]:
# Strade Tertiary

gdf_tertiary = gdf[(gdf["highway"] == "tertiary")]
len(gdf_secondary)
print(gdf[gdf["highway"] == "tertiary"].isna().sum())

gdf_tertiary = gdf[(gdf["highway"] == "tertiary") | (gdf["highway"] == "tertiary_link")]

# Salva in GeoJSON
gdf_tertiary.to_file(PATH_STRADE_TERTIARY, driver='GeoJSON')

In [ ]:
# Residential

gdf_residential = gdf[(gdf["highway"] == "residential")]
print(gdf[gdf["highway"] == "residential"].isna().sum())

print(len(gdf_secondary))  # Mostra il numero di righe

# Salva in GeoJSON
gdf_residential.to_file(PATH_STRADE_RESIDENTIAL, driver='GeoJSON')

In [ ]:
# Service

gdf_service = gdf[(gdf["highway"] == "service")]
print(gdf[gdf["highway"] == "service"].isna().sum())

print(len(gdf_service))  # Mostra il numero di righe

# Salva in GeoJSON
gdf_service.to_file(PATH_STRADE_SERVICE, driver='GeoJSON')

Forse ho cancellato il "strade_ciclabili.geojson" che stavate usando.
Dovrebbe essere questo:

In [ ]:
gdf_ciclabili_raw = gpd.read_file(PATH_CICLABILI_RAW)
gdf_ciclabili_staging = gdf_ciclabili_raw.copy()

# Convertiamo alcuni tipi
gdf_ciclabili_staging["geometry"] = gdf_ciclabili_staging["geometry"].apply(lambda x: x.geoms[0])
gdf_ciclabili_staging["fine_lavor"] = gdf_ciclabili_staging["fine_lavor"].astype(int)
gdf_ciclabili_staging["lunghezza"] = gdf_ciclabili_staging["lunghezza"].astype(int)
gdf_ciclabili_staging = gdf_ciclabili_staging.drop(["id_amat", "id_via","gerarchia"], axis=1)

gdf_ciclabili_staging = gpd.GeoDataFrame(gdf_ciclabili_staging, geometry="geometry")

e anziché caricarlo da file, ottenete il geodataframe come dizionario con la funzione .to_dict() a partire dalla prima parte di ETL del file raw che vi ho riscritto sopra.

In [ ]:
data = gdf_ciclabili_staging.to_dict()

Da ora in poi dovrebbe essere identico a prima! Non volevo disturbare il vostro lavoro ma ho ristrutturato le cartelle e i file

In [ ]:
# Creazione geodataframe manuale

# Costruisci il GeoDataFrame manualmente
features = data["features"]
geometries = [shape(feature["geometry"]) for feature in features]
properties = [feature["properties"] for feature in features]

gdf = gpd.GeoDataFrame(properties, geometry=geometries)

# Imposta il CRS (Coordinate Reference System)
gdf.set_crs("EPSG: 4326", inplace=True)

# 1. Assicuriamoci che valori None siano convertiti in np.nan
gdf['highway'] = gdf['highway'].replace({None: np.nan})

# 2. Se necessario, converti in stringa (utile se ci sono tipi misti)
gdf['highway'] = gdf['highway'].astype('object')

# 3. Crea la maschera con entrambe le condizioni:
mask = (
    gdf['highway'].isna() |
    (gdf['highway'] == 'unclassified')
)

# 4. Applica il filtro
gdf_filtered = gdf[mask]

gdf_filtered.duplicated()

gdf_filtered.to_file(PATH_highway_null_unclassified)